<a href="https://colab.research.google.com/github/ReynaraEzra/Image-Classification/blob/main/Image_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Image Classification On Rock Paper Scissors**
# **Reynara Ezra Pratama**

In [45]:
import tensorflow as tf
print(tf.__version__)

2.5.0
time: 2.45 ms (started: 2021-06-19 13:57:13 +00:00)


In [5]:
!pip install ipython-autotime
%load_ext autotime

time: 143 µs (started: 2021-06-19 10:49:29 +00:00)


## **Get Data**

In [2]:
!wget --no-check-certificate \
  https://dicodingacademy.blob.core.windows.net/picodiploma/ml_pemula_academy/rockpaperscissors.zip \
  -O /tmp/rockpaperscissors.zip

--2021-06-19 10:39:18--  https://dicodingacademy.blob.core.windows.net/picodiploma/ml_pemula_academy/rockpaperscissors.zip
Resolving dicodingacademy.blob.core.windows.net (dicodingacademy.blob.core.windows.net)... 52.239.197.36
Connecting to dicodingacademy.blob.core.windows.net (dicodingacademy.blob.core.windows.net)|52.239.197.36|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 322873683 (308M) [application/zip]
Saving to: ‘/tmp/rockpaperscissors.zip’

/tmp/rockpapersciss 100%[===================>] 307.92M  1.31MB/s    in 5m 32s  

2021-06-19 10:44:51 (950 KB/s) - ‘/tmp/rockpaperscissors.zip’ saved [322873683/322873683]



## **Extract Data**

In [6]:
import zipfile, os

local_zip = '/tmp/rockpaperscissors.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/tmp')
zip_ref.close()

time: 4.25 s (started: 2021-06-19 10:49:34 +00:00)


## **Check Data**

In [32]:
print('Number of Rock Images :{}'.format(len(os.listdir('/tmp/rockpaperscissors/rock/'))))
print('Number of Paper Images :{}'.format(len(os.listdir('/tmp/rockpaperscissors/paper/'))))
print('Number of Scissors Images :{}'.format(len(os.listdir('/tmp/rockpaperscissors/scissors/'))))
print('Total Images : {}'.format(
    len(os.listdir('/tmp/rockpaperscissors/rock/')) + 
    len(os.listdir('/tmp/rockpaperscissors/paper/')) + 
    len(os.listdir('/tmp/rockpaperscissors/scissors/'))
    )
)

Number of Rock Images :726
Number of Paper Images :712
Number of Scissors Images :750
Total Images : 2188
time: 10.7 ms (started: 2021-06-19 11:29:56 +00:00)


## **Split Folder**

In [13]:
!pip install split_folders

time: 2.43 s (started: 2021-06-19 11:20:52 +00:00)


In [14]:
import splitfolders

splitfolders.ratio('/tmp/rockpaperscissors/rps-cv-images', '/tmp/rockpaperscissors/data', seed=1, ratio=(0.6, 0.4))

Copying files: 2188 files [00:00, 4072.20 files/s]

time: 551 ms (started: 2021-06-19 11:20:59 +00:00)


In [37]:
base_dir = '/tmp/rockpaperscissors/data'
train_dir = os.path.join(base_dir, 'train') ## Train Dataset
validation_dir = os.path.join(base_dir, 'val') ## Validation Dataset

time: 1.44 ms (started: 2021-06-19 12:40:44 +00:00)


In [38]:
os.listdir('/tmp/rockpaperscissors/data/train')

['rock', 'paper', 'scissors']

time: 4.54 ms (started: 2021-06-19 12:40:46 +00:00)


In [39]:
os.listdir('/tmp/rockpaperscissors/data/val')

['rock', 'paper', 'scissors']

time: 4 ms (started: 2021-06-19 12:40:47 +00:00)


In [40]:
train_rock_dir = os.path.join(train_dir, 'rock')
train_paper_dir = os.path.join(train_dir, 'paper')
train_scissors_dir = os.path.join(train_dir, 'scissors')

validation_rock_dir = os.path.join(validation_dir, 'rock')
validation_paper_dir = os.path.join(validation_dir, 'paper')
validation_scissors_dir = os.path.join(validation_dir, 'scissors')

time: 8.92 ms (started: 2021-06-19 12:50:13 +00:00)


## **Check Train and Validation Size**

**Train 60%**

In [35]:
total_size_train = (
    len(os.listdir('/tmp/rockpaperscissors/data/train/rock')) + 
    len(os.listdir('/tmp/rockpaperscissors/data/train/paper')) +
    len(os.listdir('/tmp/rockpaperscissors/data/train/scissors'))
)
print('Total Training Data is : {}'.format(total_size_train))

Total Training Data is : 1312
time: 4.63 ms (started: 2021-06-19 11:32:21 +00:00)


**Validation 40%**

In [36]:
total_size_validation = (
    len(os.listdir('/tmp/rockpaperscissors/data/val/rock')) +
    len(os.listdir('/tmp/rockpaperscissors/data/val/paper')) +
    len(os.listdir('/tmp/rockpaperscissors/data/val/scissors'))
)
print('Total Valdation Data is : {}'.format(total_size_validation))

Total Valdation Data is : 876
time: 4.21 ms (started: 2021-06-19 11:32:39 +00:00)


## **Image Augmentation**

In [47]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
    rescale = 1./255,
    rotation_range = 20,
    horizontal_flip = True,
    shear_range = 0.2,
    zoom_range = 0.2,
    fill_mode = 'nearest'
)

validation_datagen = ImageDataGenerator(
    rescale = 1./255,
    rotation_range = 20,
    horizontal_flip = True,
    shear_range = 0.2,
    zoom_range = 0.2,
    fill_mode = 'nearest'
)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size = (150, 150),
    batch_size = 128,
    class_mode = 'categorical'
)

validation_generator = validation_datagen.flow_from_directory(
    validation_dir,
    target_size = (150, 150),
    batch_size = 128,
    class_mode = 'categorical'
)

Found 1312 images belonging to 3 classes.
Found 876 images belonging to 3 classes.
time: 219 ms (started: 2021-06-19 13:57:25 +00:00)


## **Construct Model Sequential Using CNN**

In [50]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(150, 150, 3)),
  tf.keras.layers.MaxPool2D(2,2),
  tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
  tf.keras.layers.MaxPool2D(2,2),
  tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
  tf.keras.layers.MaxPool2D(2,2),
  tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
  tf.keras.layers.MaxPool2D(2,2),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(512, activation='relu'),
  tf.keras.layers.Dense(3, activation='softmax')
])

time: 411 ms (started: 2021-06-19 14:14:36 +00:00)


In [51]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 148, 148, 32)      896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 74, 74, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 72, 72, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 36, 36, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 34, 34, 128)       73856     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 17, 17, 128)       0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 15, 15, 128)       1

## **Run Model**

In [54]:
model.compile(
    loss = 'categorical_crossentropy',
    optimizer = tf.optimizers.Adam(),
    metrics = ['accuracy']
)

time: 10.8 ms (started: 2021-06-19 14:19:15 +00:00)


In [60]:
history = model.fit(
    train_generator,
    epochs = 10,
    validation_data = validation_generator,
    verbose = 1
)

Epoch 1/10
11/11 [==============================] - 72s 7s/step - loss: 0.1281 - accuracy: 0.9581 - val_loss: 0.1147 - val_accuracy: 0.9600
Epoch 2/10
11/11 [==============================] - 72s 7s/step - loss: 0.1070 - accuracy: 0.9634 - val_loss: 0.1179 - val_accuracy: 0.9600
Epoch 3/10
11/11 [==============================] - 72s 7s/step - loss: 0.0910 - accuracy: 0.9649 - val_loss: 0.0963 - val_accuracy: 0.9669
Epoch 4/10
11/11 [==============================] - 72s 6s/step - loss: 0.0756 - accuracy: 0.9733 - val_loss: 0.0962 - val_accuracy: 0.9669
Epoch 5/10
11/11 [==============================] - 71s 6s/step - loss: 0.0884 - accuracy: 0.9688 - val_loss: 0.0768 - val_accuracy: 0.9760
Epoch 6/10
11/11 [==============================] - 72s 6s/step - loss: 0.0555 - accuracy: 0.9817 - val_loss: 0.0614 - val_accuracy: 0.9795
Epoch 7/10
11/11 [==============================] - 71s 7s/step - loss: 0.0813 - accuracy: 0.9688 - val_loss: 0.1499 - val_accuracy: 0.9463
Epoch 8/10
11/11 [==